In [ ]:
import pandas as pd

if not 'paired' in locals():
  paired = False
  raise Exception("missing argument paired (can be true or false)")

if not 'train_data_path' in locals():
  train_data_path = ""
  raise Exception("missing argument train_data_path")

if not 'test_data_path' in locals():
  test_data_path = ""
  raise Exception("missing argument test_data_path")

if not 'validation_data_path' in locals():
  validation_data_path = ""
  raise Exception("missing argument validation_data_path")

In [ ]:
# Function to verify each row
def calculate_task(row, known_epitopes, known_tcr):
  if paired:
    tcr = row['TRA_CDR3'] + '_' + row['TRB_CDR3']
  else:
    tcr = row['TRB_CDR3']

  epitope_exists = row['Epitope'] in known_epitopes
  cdr3_exists = tcr in known_tcr
  
  if epitope_exists and cdr3_exists:
    return 'TPP1'
  elif epitope_exists and not cdr3_exists:
    return 'TPP2'
  elif not epitope_exists and not cdr3_exists:
    return 'TPP3'
  elif not epitope_exists and cdr3_exists:
    return 'TPP4'
  raise Exception("Something seems wrong")  # This handles unexpected cases

In [ ]:
df_train = pd.read_csv(train_data_path, sep="\t")
df_test = pd.read_csv(test_data_path, sep="\t")
df_validation = pd.read_csv(validation_data_path, sep="\t")

df_train_val = pd.concat([df_train, df_validation])

if paired:
  df_train_val["tcr_key"] = df_train_val['TRA_CDR3'].astype(str) + '_' + df_train_val['TRB_CDR3'].astype(str)
else:
  df_train_val["tcr_key"] = df_train_val['TRB_CDR3']

seen_epitopes = set(df_train_val["Epitope"])
seen_tcr = set(df_train_val["tcr_key"])
df_test["task"] = df_test.apply(lambda x: calculate_task(x, seen_epitopes, seen_tcr), axis=1)

In [ ]:
df_test.to_csv(test_data_path, sep="\t", index=False)